In [2]:
pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 16.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor

In [37]:
# epochs = 100
# batch_size = 32
# drop_rate = 0.3

max_depth = 5
n_estimators = 5000             # 400
lr = 0.01
num_leaves = 31
early_stopping_rounds = 300      # 100
verbose = 500                  # True

In [38]:
curr_dir = os.getcwd()
curr_dir

'C:\\weather\\v_lgbm'

In [39]:
def model_train(curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_all\\"+"\\train")
    path_test = os.path.join(curr_dir+"\\data_all\\"+"\\test")
    
    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\test\\")+files)
        test = pd.concat([test, df])   
         

        
        
    X_train = train[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    X_test = test[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    y_train_LST=train[['isitu-LST']].values
    y_test_LST=test[['isitu-LST']].values
    
    y_train_TA=train[['insitu-TA']].values
    y_test_TA=test[['insitu-TA']].values
    
#     model_name = '\\save\\'+'best_model_v1.h5' 

#     filepath = os.path.join(os.getcwd()+ model_name)

#     es = EarlyStopping(monitor='val_mse', verbose=1, patience=20)

#     checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

#     callbacks = [checkpoint, es]

    lgbm_reg_LST = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_LST.fit(X_train, y_train_LST,
              eval_set=[(X_test, y_test_LST)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose)
    
    lgbm_reg_TA = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_TA.fit(X_train, y_train_TA,
              eval_set=[(X_test, y_test_TA)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose) 
    
 
    
#     lgbm_reg.load_weights(model_name)
    
    
    path_val = os.path.join(curr_dir+"\\data_all\\"+"\\validation")
    folders_val = os.listdir(path_val)
    
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\validation\\")+files)
        validation = pd.concat([validation, df])        
        
    validation_ = validation[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    prediction_LST = lgbm_reg_LST.predict(validation_)
    prediction_TA = lgbm_reg_TA.predict(validation_)
    
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(LST = prediction_LST)
    validation_partial = validation_partial.assign(TA = prediction_TA)
    
    submission=pd.read_csv('1-2_검증데이터셋.csv', encoding='utf-8')
    submission1=submission.copy()
    submission1[['isitu-LST', 'insitu-TA']] = validation_partial[['LST','TA']].values
    
    submission1.to_csv("lgbm_v1.csv", index = False)

In [40]:
model_train(curr_dir)

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[500]	valid_0's l2: 4.61078
[1000]	valid_0's l2: 4.24902
[1500]	valid_0's l2: 4.05727
[2000]	valid_0's l2: 3.9195
[2500]	valid_0's l2: 3.81373
[3000]	valid_0's l2: 3.73384
[3500]	valid_0's l2: 3.67136
[4000]	valid_0's l2: 3.62823
[4500]	valid_0's l2: 3.59384
[5000]	valid_0's l2: 3.56517
[500]	valid_0's l2: 0.429099
[1000]	valid_0's l2: 0.246352
[1500]	valid_0's l2: 0.167129
[2000]	valid_0's l2: 0.123268
[2500]	valid_0's l2: 0.0982601
[3000]	valid_0's l2: 0.0834404
[3500]	valid_0's l2: 0.0730523
[4000]	valid_0's l2: 0.0658277
[4500]	valid_0's l2: 0.0600956
[5000]	valid_0's l2: 0.0555484
